#*Shoe Classification Project*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [ ]:
import pathlib
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_dir = pathlib.Path('/content/drive/MyDrive/Dataset/train/')

image_size = (240, 240)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.25,
    height_shift_range=0.25,
    shear_range=0.2,
    zoom_range=0.25,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_data_gen = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

Found 603 images belonging to 3 classes.


In [ ]:
data_dir = pathlib.Path('/content/drive/MyDrive/Dataset/validate')

image_size = (240, 240)
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255)

validate_data_gen = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 90 images belonging to 3 classes.


In [ ]:
print("Class indices:", train_data_gen.class_indices)
print("Total number of samples:", train_data_gen.samples)
class_counts = {label: np.sum(train_data_gen.labels == label) for label in range(len(train_data_gen.class_indices))}
print("Number of images per class:", class_counts)

Class indices: {'adidas': 0, 'converse': 1, 'nike': 2}
Total number of samples: 603
Number of images per class: {0: np.int64(201), 1: np.int64(201), 2: np.int64(201)}


In [ ]:
print("Class indices:", validate_data_gen.class_indices)
print("Total number of samples:", validate_data_gen.samples)
class_counts = {label: np.sum(validate_data_gen.labels == label) for label in range(len(validate_data_gen.class_indices))}
print("Number of images per class:", class_counts)

Class indices: {'adidas': 0, 'converse': 1, 'nike': 2}
Total number of samples: 90
Number of images per class: {0: np.int64(30), 1: np.int64(30), 2: np.int64(30)}


In [ ]:
batch_images, batch_labels = next(train_data_gen)
print("Shape of batch_images (input):", batch_images.shape)
print("Shape of batch_labels (output):", batch_labels.shape)
print("First 5 labels in batch:\n", batch_labels[:5])

Shape of batch_images (input): (32, 240, 240, 3)
Shape of batch_labels (output): (32, 3)
First 5 labels in batch:
 [[1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(240, 240, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(3, activation='softmax'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=8,
    restore_best_weights=True
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=3,
    min_lr=1e-7
)
history = model.fit(
    train_data_gen,
    epochs=50,
    validation_data=validate_data_gen,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 238, 238, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 119, 119, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 117, 117, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 58, 58, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │    25,690,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,784,387 (98.36 MB)

 Trainable params: 25,784,387 (98.36 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 16s 687ms/step - accuracy: 0.2959 - loss: 1.2242 - val_accuracy: 0.3333 - val_loss: 1.0910 - learning_rate: 1.0000e-04
Epoch 2/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 11s 569ms/step - accuracy: 0.3182 - loss: 1.1002 - val_accuracy: 0.3667 - val_loss: 1.0852 - learning_rate: 1.0000e-04
Epoch 3/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 11s 576ms/step - accuracy: 0.4071 - loss: 1.0814 - val_accuracy: 0.5444 - val_loss: 1.0809 - learning_rate: 1.0000e-04
Epoch 4/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 11s 567ms/step - accuracy: 0.3667 - loss: 1.0783 - val_accuracy: 0.4222 - val_loss: 1.0722 - learning_rate: 1.0000e-04
Epoch 5/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 21s 569ms/step - accuracy: 0.3642 - loss: 1.0777 - val_accuracy: 0.4667 - val_loss: 1.0700 - learning_rate: 1.0000e-04
Epoch 6/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 11s 592ms/step - accuracy: 0.4002 - loss: 1.0833 - val_accuracy: 0.5222 - val_loss: 1.0750 - learning_rate: 1.0000e-04
Epoch 7/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 20s 579ms/step - acc

In [ ]:
model.save('final_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
loaded_model = load_model('final_model.h5')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pathlib

model = load_model('final_model.h5')

data_dir = pathlib.Path('/content/drive/MyDrive/Dataset/validate')
image_size = (240, 240)
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255)
validate_data_gen = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

loss, accuracy = model.evaluate(validate_data_gen)
print(f"\nValidation Accuracy: {accuracy*100:.2f}%")

Found 90 images belonging to 3 classes.
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 180ms/step - accuracy: 0.5695 - loss: 1.0725

Validation Accuracy: 60.00%
